# Delta Lake Optimization Project

This notebook is a guided, hands‑on lab to explore core Delta Lake optimization techniques in Databricks. You'll iteratively: generate data → profile baseline performance → apply optimizations → observe impact in the Spark UI and table metadata.

## Learning Objectives
By the end you should be able to:
- Select appropriate physical layout strategies (partitioning, Z-Ordering, Liquid Clustering)
- Diagnose small-file and data-skipping issues
- Apply manual vs automatic compaction
- Use VACUUM safely for storage hygiene
- Read/interpret Spark UI metrics (files scanned, data read, predicate pushdown, scan time)

## Flow Overview
1. Environment + table name registry
2. Data generation (idempotent)
3. Baseline query + metrics capture checklist
4. Partitioning impact
5. Z-Ordering for multi-dimensional skipping
6. Manual vs auto compaction
7. Liquid Clustering (adaptive layout)
8. VACUUM lifecycle management
9. Cleanup

> Tip: After each transformation, open the Spark UI and record: files scanned, total data read MB, and wall-clock time. Building a mini table of results reinforces the concepts.

## Setup
We first define the catalog & schema (database) and then a centralized registry of logical table names to keep code DRY and readable. Ensure your workspace permissions allow catalog/schema creation.

In [0]:
# Configuration
CATALOG_NAME = "delta_optimization_project"
SCHEMA_NAME = "sales_data"

# Create the catalog and schema if they don't exist
spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG_NAME}")
spark.sql(f"USE CATALOG {CATALOG_NAME}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME}")
spark.sql(f"USE SCHEMA {SCHEMA_NAME}")

In [0]:
# Centralized table name registry & helper utilities
TABLES = {
    "raw": f"{CATALOG_NAME}.{SCHEMA_NAME}.sales_raw",
    "partitioned": f"{CATALOG_NAME}.{SCHEMA_NAME}.sales_partitioned",
    "raw_zorder": f"{CATALOG_NAME}.{SCHEMA_NAME}.sales_raw_zorder",
    "to_compact": f"{CATALOG_NAME}.{SCHEMA_NAME}.sales_to_compact",
    "auto_compact": f"{CATALOG_NAME}.{SCHEMA_NAME}.sales_auto_compact",
    "liquid_clustered": f"{CATALOG_NAME}.{SCHEMA_NAME}.sales_liquid_clustered",
}

def tbl(key: str) -> str:
    """Return fully qualified table name by logical key."""
    return TABLES[key]

from typing import Optional
import pyspark.sql.functions as F

## Step 1: Generate Synthetic Sales Data

We create a moderately large, semi-realistic sales fact dataset to exercise partition pruning and data skipping. Characteristics:
- 5M rows (adjustable) to give enough files for optimization to matter
- Mixed cardinalities: `country` (low), `customer_id` (medium/high), `product_id` (medium) for contrasting layout strategies
- Temporal column `sale_date` enabling potential future partitioning or clustering experiments

### Design Notes
- Data generation is idempotent: if the base table exists we skip regeneration (saves cluster time)
- We intentionally over-partition the initial write (`num_partitions` + low `maxRecordsPerFile`) to create many small files and highlight gains from compaction

### Your Tasks
1. Run the cell.
2. If generation runs, note: number of partitions written, approximate file count (later visible via `DESCRIBE DETAIL` or storage listing).
3. If skipped, confirm reuse message printed.

Proceed once the table exists.

In [0]:
# Generate synthetic data at scale using Spark (executor-friendly, no pandas/Faker)
import pyspark.sql.functions as F

# If the target table already exists, skip expensive generation and reuse it.
# This makes the notebook idempotent for iterative runs.
raw_exists = spark.sql(f"SHOW TABLES IN {CATALOG_NAME}.{SCHEMA_NAME} LIKE 'sales_raw'").count() > 0
if raw_exists:
    print(f"Table {tbl('raw')} already exists; skipping data generation.")
else:
    # Number of records to generate (adjust for your cluster)
    num_records = 5_000_000

    # Controls the number of output files (one file per output partition, roughly).
    # Increase partitions to create more small files; decrease to reduce file count.
    num_partitions = 1000

    # Small list of countries used for sampling (kept on driver only)
    countries = ['United States','United Kingdom','Germany','France','Canada','Australia','India','Brazil','Japan','Netherlands']

    # Build the DataFrame using Spark primitives (scales across executors)
    df = (
        spark.range(num_records)
        .repartition(num_partitions)
        .withColumnRenamed('id', 'seq')
        .withColumn(
            'transaction_id',
            F.concat(
                F.lit('txn_'),
                F.col('seq').cast('string'),
                F.lit('_'),
                # cast the double returned by rand() to string before md5 to avoid datatype mismatch
                F.substring(F.md5(F.rand(12345).cast('string')), 1, 8)
            )
        )
        .withColumn('customer_id', (F.floor(F.rand(42) * 1001) + 1000).cast('int'))
        .withColumn('product_id', (F.floor(F.rand(99) * 401) + 100).cast('int'))
        .withColumn('sale_date', F.date_sub(F.current_date(), F.floor(F.rand(7) * 730).cast('int')))
        .withColumn('quantity', (F.floor(F.rand(11) * 10) + 1).cast('int'))
        .withColumn('unit_price', F.round(F.rand(13) * 190.0 + F.lit(10.5), 2))
        .withColumn(
            'country',
            F.element_at(
                F.array(*[F.lit(c) for c in countries]),
                (F.floor(F.rand(21) * len(countries)) + 1).cast('int')
            )
        )
    )
    # Write the data to a Delta table. Use maxRecordsPerFile to force small files in a performant, distributed write.
    (df.write
       .format('delta')
       .option('maxRecordsPerFile', 5000)
       .mode('overwrite')
       .saveAsTable(tbl('raw'))
    )

In [0]:
df_raw = spark.read.table(tbl('raw'))

## Step 2: Baseline Query (Unoptimized Table)

We establish a performance baseline before altering layout. This isolates the effect of later optimizations.

### Query Pattern
Filter on a specific `(customer_id, product_id)` pair. This mimics a targeted lookup often seen in downstream analytics or service patterns.

### What to Capture (Create a simple log table for yourself)
| Metric | Value | How to Find |
|--------|-------|-------------|
| Files scanned | ? | Spark UI -> SQL/Dataframe -> Scan node |
| Bytes read | ? | Spark UI -> Scan details |
| Duration (s) | ? | Job / Stage timeline |
| Output rows | ? | DataFrame action output |

1. Run the cell below.

In [0]:
# Baseline query — read from saved Delta table
from pyspark.sql.functions import col

df_raw.where((col("customer_id") == 1500) & (col("product_id") == 250)).show()

2. Open Spark UI while it is/after it finishes.
3. Record metrics above.
4. Keep this snapshot; you'll compare after each optimization layer.

## Step 3: Implementing Partitioning

Partitioning physically groups data by directory on chosen columns. Pros: efficient directory pruning when filters match partition predicates. Cons: too many partitions => small files & metadata overhead; high-cardinality partition keys are an anti-pattern.

We choose `country` (low cardinality ~10) as a demonstrative partition column. Even though our query predicate currently does *not* include `country`, observing the plan difference illustrates when partitioning helps or is neutral.

### Good Candidates
- Low to moderate cardinality (10s–100s distinct) used frequently in filters
- Dimension-style attributes (e.g., region, date (at coarse grain), category)

### Avoid
- High-cardinality (user ids, transaction ids)
- Rapidly growing distinct sets (timestamps to the second)

Create a partitioned table

In [0]:
(df_raw.write.format("delta")
 .mode("overwrite")
 .partitionBy("country")
 .saveAsTable(tbl('partitioned')))

Run the same query on the partitioned table

In [0]:
(spark.read.table(tbl('partitioned'))
      .where((col("customer_id") == 1500) & (col("product_id") == 250))
      .show())

**Analysis of Partitioning:**

Go to the Spark UI and compare the execution plan with the baseline. You'll notice that if a filter on the partition key (`country`) was present, Spark would be able to prune entire directories, significantly reducing the amount of data scanned. Even without a direct filter on the partition key, observe any changes in the query plan.

## Step 4: Applying Z-Ordering

Z-Ordering re-writes data files using a multi-dimensional space-filling curve (Z-order) so rows with similar values for selected columns co-locate. This increases data skipping efficiency without exploding directory counts like over-partitioning.

### When to Use
- Repeated predicates on 1–4 moderate/high cardinality columns
- Mixed access patterns (can't commit to a single partition key)
- Need to cluster on columns that would be poor partitions

### Column Choice Tips
| Scenario | Better for Partition | Better for Z-Order |
|----------|---------------------|--------------------|
| Very low cardinality | Yes | Unnecessary |
| High cardinality | No | Often yes |
| Evolving query set | Maybe (limited) | Yes |

We Z-Order on `customer_id`, `product_id`.

Create Z-Ordered copy of raw table.

In [0]:
from delta.tables import *

spark.sql(f"DROP TABLE IF EXISTS {tbl('raw_zorder')}")

# Create a separate table from sales_raw to apply Z-Ordering to (keeps sales_partitioned untouched)
spark.sql(f"CREATE TABLE {tbl('raw_zorder')} USING DELTA AS SELECT * FROM {tbl('raw')}")

# Convert the new table to a DeltaTable object
delta_table = DeltaTable.forName(spark, tbl('raw_zorder'))

# Apply Z-Ordering on high-cardinality columns for the raw copy
delta_table.optimize().executeZOrderBy("customer_id", "product_id")

Rerun the query to see the effect of Z-Ordering on the z-ordered raw table

In [0]:
(spark.read.table(tbl('raw_zorder'))
      .where((col("customer_id") == 1500) & (col("product_id") == 250))
      .show())

**Analysis of Z-Ordering:**

In the Spark UI, look at the "Details" for the scan phase. You should see a significant reduction in the number of files read, demonstrating the effectiveness of data skipping.

## Step 5: Manual Compaction (OPTIMIZE)

Small files inflate job overhead (task scheduling, metadata reads, open/close costs) and degrade predicate pushdown (more file headers to scan). `OPTIMIZE` rewrites many small files into fewer larger ones (typically 256MB target on Databricks unless overridden).

### Simulation
We append tiny random samples repeatedly to manufacture fragmentation.

> Rule of thumb: Target 100–500MB per file for large analytical tables (depends on workload).

Run loop to append small data.

In [0]:
# Simulate small file creation by writing in a loop
for i in range(10):
    (df_raw.sample(fraction=0.001)
     .write.format("delta")
     .mode("append")
     .saveAsTable(tbl('to_compact')))

List number of files

In [0]:
# Check the number of files
spark.sql(f"DESCRIBE DETAIL {tbl('to_compact')}").select("numFiles").show()

Execute `OPTIMIZE` (compaction only — no Z-order here).

In [0]:
# Now, perform manual compaction
delta_table_to_compact = DeltaTable.forName(spark, tbl('to_compact'))
delta_table_to_compact.optimize().executeCompaction()

Re-list object count

In [0]:
# Check the number of files
spark.sql(f"DESCRIBE DETAIL {tbl('to_compact')}").select("numFiles").show()

**Analysis of Compaction:**

- Observe the reduction in the number of files after running `OPTIMIZE`. This leads to more efficient reads as Spark needs to open and process fewer files.

## Step 6: Auto Compaction

Auto Optimize (optimizeWrites + autoCompact) reduces operational toil. It sizes files during write and schedules asynchronous compaction of lingering small files.

### What It Does
- optimizeWrite: coalesces output at write time to hit target size
- autoCompact: background job merges residual small files post-commit

### Caveats
- Not a substitute for strategic Z-Ordering or Liquid Clustering
- Background lag: immediate file count after write may still be high briefly

Create table with properties enabled.


In [0]:
# Create a table with auto-compaction enabled
spark.sql(f"""
CREATE OR REPLACE TABLE {tbl('auto_compact')}
USING DELTA
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true'
)
AS SELECT * FROM {tbl('raw')}
""")


Perform small appends.

In [0]:

# Simulate small writes to this table
for i in range(10):
    (df_raw.sample(fraction=0.001)
     .write.format("delta")
     .mode("append")
     .saveAsTable(tbl('auto_compact')))

**Analysis of Auto Compaction:**

- After the writes complete, query the table's history to see the `OPTIMIZE` operations that were automatically triggered. Auto-compaction helps maintain optimal file sizes without manual intervention.

In [0]:
display(spark.sql(f"DESCRIBE HISTORY {tbl('auto_compact')}"))

## Step 7: Liquid Clustering

Liquid Clustering decouples logical clustering from static partitions. Databricks can dynamically maintain clustering as data evolves without rigid directory hierarchies.

### Advantages vs Partitioning/Z-Order
- Adaptive: automatically reorganizes incremental data
- Multi-column "soft" clustering without explosion in partitions
- Plays well with schema evolution and changing query shapes

### When to Prefer
- High churn / streaming upserts
- Many predicates across overlapping column sets
- Avoiding maintenance burden of periodic manual Z-Order runs

> Consider long-term ops: Which technique minimizes recurring maintenance for your workload?

Create clustered table and load data.

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {tbl('liquid_clustered')} (
  seq LONG,
  transaction_id STRING,
  customer_id INT,
  product_id INT,
  sale_date DATE,
  quantity INT,
  unit_price DOUBLE,
  country STRING
)
USING DELTA
CLUSTER BY (customer_id, product_id)
""")

# Insert data into the clustered table
df_raw.write.format("delta").mode("overwrite").saveAsTable(tbl('liquid_clustered'))

Run predicate query; record scan metrics. Compare with Z-Order table results.

In [0]:
# Rerun the query on the liquid clustered table
(spark.read.table(tbl('liquid_clustered'))
      .where((col("customer_id") == 1500) & (col("product_id") == 250))
      .show())

**Analysis of Liquid Clustering:**

1.  Run the query on the liquid clustered table.
2.  Examine the Spark UI. Liquid Clustering provides similar data skipping benefits as Z-Ordering but is more adaptive to changes in your data and queries over time.

## Step 8: VACUUM

`VACUUM` removes obsolete files no longer referenced by the Delta transaction log (old snapshots, overwritten data). This reclaims storage and can reduce metadata overhead.

### Safety Mechanism
Default retention (168h / 7d) protects against readers holding older snapshots (time travel) and against eventual consistency edge cases.

### Demo Adjustment
We disable retention checks only for demonstration to show effect immediately — *never do this in production* unless you fully understand snapshot isolation & streaming lag impacts.

> Production Practice: Keep retention ≥ 7 days; align with compliance & restore objectives.

### Perform updates/deletes before VACUUM to create orphaned files.

In [0]:
# Example update: Increase quantity by 10 for a specific customer
spark.sql(f"""
UPDATE {tbl('partitioned')}
SET quantity = quantity + 10
WHERE customer_id >= 1500
""")

# Example delete: Remove records for a specific product
spark.sql(f"""
DELETE FROM {tbl('partitioned')}
WHERE product_id <= 250
""")

Check number of files before vacuum

In [0]:
spark.sql(f"DESCRIBE DETAIL {tbl('partitioned')}").select("numFiles").show()

## Run VACUUM

> **Note:**  
> In Databricks Free Edition, you cannot disable the `retentionDurationCheck` safety feature.  
> This means you may need to wait up to **7 days** after performing update or delete operations before orphaned files can be removed by VACUUM.

In [0]:
# Disable the retention check
#spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "true")

# VACUUM the partitioned table
delta_table_to_vacuum = DeltaTable.forName(spark, tbl('partitioned'))
delta_table_to_vacuum.vacuum()

# Re-enable the retention check
#spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "true")

Check number of files after vacuum

In [0]:
spark.sql(f"DESCRIBE DETAIL {tbl('partitioned')}").select("numFiles").show()

**Analysis of VACUUM:**

- The `VACUUM` command cleans up files that are no longer part of the current version of the table. You can check the file system before and after running `VACUUM` on a table that has undergone several modifications to see the effect.

## Project Conclusion

You applied multiple, complementary Delta optimization techniques and observed their impact empirically.

### Comparative Summary (Qualitative)
| Technique | Primary Benefit | Ideal Use Case | Ongoing Maintenance |
|-----------|-----------------|----------------|---------------------|
| Partitioning | Directory pruning | Low-cardinality, common filters | Low once chosen |
| Z-Ordering | Multi-column data skipping | Repeated predicates across moderate/high-card columns | Periodic re-run as data grows |
| Manual Compaction | Fewer larger files | Backfill / bursty small writes | Run as needed (monitor file size dist) |
| Auto Optimize | Reduce small files proactively | Continuous ingestion workloads | Minimal |
| Liquid Clustering | Adaptive layout | Dynamic predicates & evolving schemas | Managed (monitor clustering metrics) |
| VACUUM | Storage reclamation | Any table with churn | Scheduled (respect retention) |

### Key Heuristics
- Start simple: partition only when clear benefit.
- Use Z-Order or Liquid Clustering to refine skipping without over-partitioning.
- Monitor file counts & average file size (DESCRIBE DETAIL) for compaction signals.
- Keep time travel / retention policies aligned with data recovery requirements.

### Suggested Next Experiments
1. Add a date-based partition layer and compare with country partitioning.
2. Introduce updates/deletes & measure impact on small file generation.
3. Benchmark query patterns with and without Z-Order refresh after significant data growth.
4. Use Photon runtime and compare scan metrics.
5. Track metrics programmatically and plot improvements over time.

## Cleanup
Run the following cell to drop the catalog and all associated tables created during this project.

In [0]:
# Drop the catalog and all its contents
# spark.sql(f"DROP CATALOG IF EXISTS {CATALOG_NAME} CASCADE")